In [20]:
import pandas_datareader.data as web #to collect data
import datetime as dt #to specify start and end dates

# import yfinance as yf

import eventstudy as es
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


import pandas as pd

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.regression.rolling import RollingOLS

from patsy import dmatrices
from tqdm.notebook import tqdm
tqdm.pandas()

pd.set_option('display.max_columns', 500)

## Data Reading

In [2]:
ols120 = pd.read_csv("ols120.csv", index_col = 0)

In [3]:
ols120["AsOnDate"] = pd.to_datetime(ols120["AsOnDate"], format = "%Y-%m-%d")

## 120 CAR Calc

In [4]:
ols120CAR = ols120.sort_values(by = ["CompanyName", "AsOnDate"])
ols120CAR

,CompanyName,AsOnDate,ACP,pct,RF,RMRF,MF,SMB,HML,OLS120_intercept,OLS120_RMRF,OLS120_SMB,OLS120_HML,OLS120_r_squared,OLS120_adjusted_r_squared,OLS120_f_p_value
0,20 Microns Ltd.,2008-10-06,16.82,NaN,0.069713,-6.381449,-6.311735,-0.373052,-0.566450,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20 Microns Ltd.,2008-10-07,15.05,-0.105232,0.023232,-0.669144,-0.645911,-1.502487,0.184699,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20 Microns Ltd.,2008-10-08,13.25,-0.119601,0.023232,-3.533362,-3.510130,-1.780674,0.072932,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20 Microns Ltd.,2008-10-10,11.60,-0.124528,0.045520,-7.052324,-7.006804,0.217126,0.354629,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20 Microns Ltd.,2008-10-13,12.32,0.062069,0.066863,5.042738,5.109602,-2.437097,0.145853,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13465088,Zylog Systems Ltd.,2024-03-21,0.35,0.000000,0.018215,1.441807,1.460021,0.572866,1.270303,-0.026702,0.003907,0.003905,-0.001484,0.049581,0.025001,0.115362
13465089,Zylog Systems Ltd.,2024-03-22,0.35,0.000000,0.018215,0.593324,0.611538,0.752002,0.124527,-0.026628,0.004073,0.003156,-0.001910,0.044392,0.019678,0.151766
13465090,Zylog Systems Ltd.,2024-03-26,0.35,0.000000,0.072878,-0.044514,0.028364,-1.091597,0.440605,-0.026633,0.004079,0.003177,-0.001971,0.045188,0.020494,0.145564
13465091,Zylog Systems Ltd.,2024-03-27,0.35,0.000000,0.018215,0.326702,0.344916,-0.165072,-0.188505,-0.026979,0.004395,0.004305,-0.001647,0.062475,0.038228,0.057167


In [5]:
# FUNCTION

# def CAR120(frame):
#     if ((~frame.OLS120_intercept.isnull().any()) == True):
#         ER = frame.RF + (frame.RMRF * frame.OLS120_RMRF) + (frame.SMB * frame.OLS120_SMB) + (frame.HML * frame.OLS120_HML) + frame.OLS120_intercept
#         AR = frame.pct - ER
#         return AR.sum()
#     else: 
#         return np.NaN

    # for i in range(len(frame)):
    #     if (i>0) & ( i< len(frame)-1) :
    #         coeff120CAR = np.array([frame.iloc[i]["OLS120_RMRF"], frame.iloc[i]["OLS120_SMB"],
    #                                 frame.iloc[i]["OLS120_HML"], 1, frame.iloc[i]["OLS120_intercept"]])

    #         event120CAR3 = np.array(frame.loc[ [i-1, i, i+1], ["RMRF", "SMB", "HML", "RF"]],)
    #         event120CAR3 = [np.append(item, 1) for item in event120CAR3]

    #         ER = np.matmul(event120CAR3, coeff120CAR)

    #         actualReturns = np.array([frame.iloc[i-1]["pct"], frame.iloc[i]["pct"], frame.iloc[i+1]["pct"]])

    #         AR = np.subtract( actualReturns, ER )

    #         output120CAR3.loc[i, ["120CAR3"]] = AR.sum()


    
        # if i>=2:
        #     frame["120CAR5"].iloc[i] = CAR120(frame.iloc[ i-2 : i+2 ])
        # if i>=3:
        #     frame["120CAR7"].iloc[i] = CAR120(frame.iloc[ i-3 : i+3 ])
        # if i>=5:
        #     frame["120CAR11"].iloc[i] = CAR120(frame.iloc[ i-5 : i+5 ])

        

def CAR(frame):

    outputFrame = pd.DataFrame( index = frame.index)

        
    # 3 Days
    
    outputFrame["pct_cen3sum"] = frame["pct"].rolling(window = 3, min_periods = 3, center = True).sum()
    
    outputFrame["RF_cen3sum"] = frame["RF"].rolling(window = 3, min_periods = 3, center = True).sum()
    
    outputFrame["RMRF_cen3sum"] = frame["RMRF"].rolling(window = 3, min_periods = 3, center = True).sum()
    outputFrame["SMB_cen3sum"] = frame["SMB"].rolling(window = 3, min_periods = 3, center = True).sum()
    outputFrame["HML_cen3sum"] = frame["HML"].rolling(window = 3, min_periods = 3, center = True).sum()
    
    
    # 5 Days
    
    outputFrame["pct_cen5sum"] = frame["pct"].rolling(window = 5, min_periods = 5, center = True).sum()
    
    outputFrame["RF_cen5sum"] = frame["RF"].rolling(window = 5, min_periods = 5, center = True).sum()
    
    outputFrame["RMRF_cen5sum"] = frame["RMRF"].rolling(window = 5, min_periods = 5, center = True).sum()
    outputFrame["SMB_cen5sum"] = frame["SMB"].rolling(window = 5, min_periods = 5, center = True).sum()
    outputFrame["HML_cen5sum"] = frame["HML"].rolling(window = 5, min_periods = 5, center = True).sum()
    
    
    # 7 Days
    
    outputFrame["pct_cen7sum"] = frame["pct"].rolling(window = 7, min_periods = 7, center = True).sum()
    
    outputFrame["RF_cen7sum"] = frame["RF"].rolling(window = 7, min_periods = 7, center = True).sum()
    
    outputFrame["RMRF_cen7sum"] = frame["RMRF"].rolling(window = 7, min_periods = 7, center = True).sum()
    outputFrame["SMB_cen7sum"] = frame["SMB"].rolling(window = 7, min_periods = 7, center = True).sum()
    outputFrame["HML_cen7sum"] = frame["HML"].rolling(window = 7, min_periods = 7, center = True).sum()
    
    
    # 11 Days
    
    outputFrame["pct_cen11sum"] = frame["pct"].rolling(window = 11, min_periods = 11, center = True).sum()
    
    outputFrame["RF_cen11sum"] = frame["RF"].rolling(window = 11, min_periods = 11, center = True).sum()
    
    outputFrame["RMRF_cen11sum"] = frame["RMRF"].rolling(window = 11, min_periods = 11, center = True).sum()
    outputFrame["SMB_cen11sum"] = frame["SMB"].rolling(window = 11, min_periods = 11, center = True).sum()
    outputFrame["HML_cen11sum"] = frame["HML"].rolling(window = 11, min_periods = 11, center = True).sum()
    
    
    return outputFrame


In [6]:
result120CAR = ols120CAR.sort_values(by = ["CompanyName", "AsOnDate"]).groupby(by = "CompanyName").progress_apply(CAR)

  0%|          | 0/3721 [00:00<?, ?it/s]

In [7]:
result120CAR

pct_cen3sum  RF_cen3sum  RMRF_cen3sum  \
CompanyName                                                          
20 Microns Ltd.    0                 NaN         NaN           NaN   
                   1                 NaN    0.116178    -10.583954   
                   2           -0.349361    0.091985    -11.254830   
                   3           -0.182061    0.135616     -5.542948   
                   4           -0.163920    0.134667     -0.834193   
...                                  ...         ...           ...   
Zylog Systems Ltd. 13465088     0.000000    0.054643      2.222872   
                   13465089     0.000000    0.109307      1.990617   
                   13465090     0.000000    0.109307      0.875511   
                   13465091     0.000000    0.109307      0.985242   
                   13465092          NaN         NaN           NaN   

                             SMB_cen3sum  HML_cen3sum  pct_cen5sum  \
CompanyName                                                          
20 Microns Ltd.    0                 NaN          NaN          NaN   
                   1           -3.656212    -0.308820          NaN   
                   2           -3.066034     0.612259          NaN   
                   3           -4.000644     0.573414    -0.388754   
                   4           -1.442626     1.579289    -0.183251   
...                                  ...          ...          ...   
Zylog Systems Ltd. 13465088     0.647351     1.691011     0.000000   
                   13465089     0.233272     1.835435     0.000000   
                   13465090    -0.504666     0.376627     0.000000   
                   13465091    -1.836003     0.159099          NaN   
                   13465092          NaN          NaN          NaN   

                             RF_cen5sum  RMRF_cen5sum  SMB_cen5sum  \
CompanyName                                                          
20 Microns Ltd.    0                NaN           NaN          NaN   
                   1                NaN           NaN          NaN   
                   2           0.228562    -12.593540    -5.876183   
                   3           0.181132     -5.036699    -4.725787   
                   4           0.180182     -9.543586    -3.464452   
...                                 ...           ...          ...   
Zylog Systems Ltd. 13465088    0.145734      0.669402    -0.213495   
                   13465089    0.145735      2.505059    -0.609317   
                   13465090    0.145736      3.020372    -0.511135   
                   13465091         NaN           NaN          NaN   
                   13465092         NaN           NaN          NaN   

                             HML_cen5sum  pct_cen7sum  RF_cen7sum  \
CompanyName                                                         
20 Microns Ltd.    0                 NaN          NaN         NaN   
                   1                 NaN          NaN         NaN   
                   2            0.191663          NaN         NaN   
                   3            1.836919          NaN    0.273128   
                   4            1.702691    -0.163688    0.225697   
...                                  ...          ...         ...   
Zylog Systems Ltd. 13465088     1.701467     0.000000    0.218599   
                   13465089     1.943111     0.000000    0.182163   
                   13465090     1.553929          NaN         NaN   
                   13465091          NaN          NaN         NaN   
                   13465092          NaN          NaN         NaN   

                             RMRF_cen7sum  SMB_cen7sum  HML_cen7sum  \
CompanyName                                                           
20 Microns Ltd.    0                  NaN          NaN          NaN   
                   1                  NaN          NaN          NaN   
                   2                  NaN          NaN          NaN   
                   3           -16.594178  

In [8]:
result120CAR2 = result120CAR.reset_index(level = 0).drop("CompanyName", axis = 1)
result120CAR2

,pct_cen3sum,RF_cen3sum,RMRF_cen3sum,SMB_cen3sum,HML_cen3sum,pct_cen5sum,RF_cen5sum,RMRF_cen5sum,SMB_cen5sum,HML_cen5sum,pct_cen7sum,RF_cen7sum,RMRF_cen7sum,SMB_cen7sum,HML_cen7sum,pct_cen11sum,RF_cen11sum,RMRF_cen11sum,SMB_cen11sum,HML_cen11sum
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,0.116178,-10.583954,-3.656212,-0.308820,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.349361,0.091985,-11.254830,-3.066034,0.612259,NaN,0.228562,-12.593540,-5.876183,0.191663,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-0.182061,0.135616,-5.542948,-4.000644,0.573414,-0.388754,0.181132,-5.036699,-4.725787,1.836919,NaN,0.273128,-16.594178,-5.339991,1.320939,NaN,NaN,NaN,NaN,NaN
4,-0.163920,0.134667,-0.834193,-1.442626,1.579289,-0.183251,0.180182,-9.543586,-3.464452,1.702691,-0.163688,0.225697,-12.320480,-5.095004,1.701136,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13465088,0.000000,0.054643,2.222872,0.647351,1.691011,0.000000,0.145734,0.669402,-0.213495,1.701467,0.000000,0.218599,1.086124,-0.832943,1.912107,NaN,NaN,NaN,NaN,NaN
13465089,0.000000,0.109307,1.990617,0.233272,1.835435,0.000000,0.145735,2.505059,-0.609317,1.943111,0.000000,0.182163,1.699158,-0.957902,1.419960,NaN,NaN,NaN,NaN,NaN
13465090,0.000000,0.109307,0.875511,-0.504666,0.376627,0.000000,0.145736,3.020372,-0.511135,1.553929,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13465091,0.000000,0.109307,0.985242,-1.836003,0.159099,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
output120CAR = pd.concat([ols120CAR, result120CAR2], axis = 1)
output120CAR

,CompanyName,AsOnDate,ACP,pct,RF,RMRF,MF,SMB,HML,OLS120_intercept,OLS120_RMRF,OLS120_SMB,OLS120_HML,OLS120_r_squared,OLS120_adjusted_r_squared,OLS120_f_p_value,pct_cen3sum,RF_cen3sum,RMRF_cen3sum,SMB_cen3sum,HML_cen3sum,pct_cen5sum,RF_cen5sum,RMRF_cen5sum,SMB_cen5sum,HML_cen5sum,pct_cen7sum,RF_cen7sum,RMRF_cen7sum,SMB_cen7sum,HML_cen7sum,pct_cen11sum,RF_cen11sum,RMRF_cen11sum,SMB_cen11sum,HML_cen11sum
0,20 Microns Ltd.,2008-10-06,16.82,NaN,0.069713,-6.381449,-6.311735,-0.373052,-0.566450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20 Microns Ltd.,2008-10-07,15.05,-0.105232,0.023232,-0.669144,-0.645911,-1.502487,0.184699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.116178,-10.583954,-3.656212,-0.308820,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20 Microns Ltd.,2008-10-08,13.25,-0.119601,0.023232,-3.533362,-3.510130,-1.780674,0.072932,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.349361,0.091985,-11.254830,-3.066034,0.612259,NaN,0.228562,-12.593540,-5.876183,0.191663,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20 Microns Ltd.,2008-10-10,11.60,-0.124528,0.045520,-7.052324,-7.006804,0.217126,0.354629,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.182061,0.135616,-5.542948,-4.000644,0.573414,-0.388754,0.181132,-5.036699,-4.725787,1.836919,NaN,0.273128,-16.594178,-5.339991,1.320939,NaN,NaN,NaN,NaN,NaN
4,20 Microns Ltd.,2008-10-13,12.32,0.062069,0.066863,5.042738,5.109602,-2.437097,0.145853,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.163920,0.134667,-0.834193,-1.442626,1.579289,-0.183251,0.180182,-9.543586,-3.464452,1.702691,-0.163688,0.225697,-12.320480,-5.095004,1.701136,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13465088,Zylog Systems Ltd.,2024-03-21,0.35,0.000000,0.018215,1.441807,1.460021,0.572866,1.270303,-0.026702,0.003907,0.003905,-0.001484,0.049581,0.025001,0.115362,0.000000,0.054643,2.222872,0.647351,1.691011,0.000000,0.145734,0.669402,-0.213495,1.701467,0.000000,0.218599,1.086124,-0.832943,1.912107,NaN,NaN,NaN,NaN,NaN
13465089,Zylog Systems Ltd.,2024-03-22,0.35,0.000000,0.018215,0.593324,0.611538,0.752002,0.124527,-0.026628,0.004073,0.003156,-0.001910,0.044392,0.019678,0.151766,0.000000,0.109307,1.990617,0.233272,1.835435,0.000000,0.145735,2.505059,-0.609317,1.943111,0.000000,0.182163,1.699158,-0.957902,1.419960,NaN,NaN,NaN,NaN,NaN
13465090,Zylog Systems Ltd.,2024-03-26,0.35,0.000000,0.072878,-0.044514,0.028364,-1.091597,0.440605,-0.026633,0.004079,0.003177,-0.001971,0.045188,0.020494,0.145564,0.000000,0.109307,0.875511,-0.504666,0.376627,0.000000,0.145736,3.020372,-0.511135,1.553929,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13465091,Zylog Systems Ltd.,2024-03-27,0.35,0.000000,0.018215,0.326702,0.344916,-0.165072,-0.188505,-0.026979,0.004395,0.004305,-0.001647,0.062475,0.038228,0.057167,0.000000,0.109307,0.985242,-1.836003,0.159099,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
output120CAR.columns

Index(['CompanyName', 'AsOnDate', 'ACP', 'pct', 'RF', 'RMRF', 'MF', 'SMB',
       'HML', 'OLS120_intercept', 'OLS120_RMRF', 'OLS120_SMB', 'OLS120_HML',
       'OLS120_r_squared', 'OLS120_adjusted_r_squared', 'OLS120_f_p_value',
       'pct_cen3sum', 'RF_cen3sum', 'RMRF_cen3sum', 'SMB_cen3sum',
       'HML_cen3sum', 'pct_cen5sum', 'RF_cen5sum', 'RMRF_cen5sum',
       'SMB_cen5sum', 'HML_cen5sum', 'pct_cen7sum', 'RF_cen7sum',
       'RMRF_cen7sum', 'SMB_cen7sum', 'HML_cen7sum', 'pct_cen11sum',
       'RF_cen11sum', 'RMRF_cen11sum', 'SMB_cen11sum', 'HML_cen11sum'],
      dtype='object')

In [11]:
output120CAR["120CAR3"] = output120CAR["pct_cen3sum"] - (output120CAR["RF_cen3sum"] + 
                          
                          output120CAR["OLS120_RMRF"]*output120CAR["RMRF_cen3sum"] + 
                          output120CAR["OLS120_SMB"]*output120CAR["SMB_cen3sum"] + 
                          output120CAR["OLS120_HML"]*output120CAR["HML_cen3sum"] + 
                          output120CAR["OLS120_intercept"]*3 )


In [12]:
output120CAR["120CAR5"] = output120CAR["pct_cen5sum"] - (output120CAR["RF_cen5sum"] + 
                          
                          output120CAR["OLS120_RMRF"]*output120CAR["RMRF_cen5sum"] + 
                          output120CAR["OLS120_SMB"]*output120CAR["SMB_cen5sum"] + 
                          output120CAR["OLS120_HML"]*output120CAR["HML_cen5sum"] + 
                          output120CAR["OLS120_intercept"]*5 )

In [13]:
output120CAR["120CAR7"] = output120CAR["pct_cen7sum"] - (output120CAR["RF_cen7sum"] + 
                          
                          output120CAR["OLS120_RMRF"]*output120CAR["RMRF_cen7sum"] + 
                          output120CAR["OLS120_SMB"]*output120CAR["SMB_cen7sum"] + 
                          output120CAR["OLS120_HML"]*output120CAR["HML_cen7sum"] + 
                          output120CAR["OLS120_intercept"]*7 )

In [14]:
output120CAR["120CAR11"] = output120CAR["pct_cen11sum"] - (output120CAR["RF_cen11sum"] + 
                          
                          output120CAR["OLS120_RMRF"]*output120CAR["RMRF_cen11sum"] + 
                          output120CAR["OLS120_SMB"]*output120CAR["SMB_cen11sum"] + 
                          output120CAR["OLS120_HML"]*output120CAR["HML_cen11sum"] + 
                          output120CAR["OLS120_intercept"]*11 )

In [15]:
output120CAR

,CompanyName,AsOnDate,ACP,pct,RF,RMRF,MF,SMB,HML,OLS120_intercept,OLS120_RMRF,OLS120_SMB,OLS120_HML,OLS120_r_squared,OLS120_adjusted_r_squared,OLS120_f_p_value,pct_cen3sum,RF_cen3sum,RMRF_cen3sum,SMB_cen3sum,HML_cen3sum,pct_cen5sum,RF_cen5sum,RMRF_cen5sum,SMB_cen5sum,HML_cen5sum,pct_cen7sum,RF_cen7sum,RMRF_cen7sum,SMB_cen7sum,HML_cen7sum,pct_cen11sum,RF_cen11sum,RMRF_cen11sum,SMB_cen11sum,HML_cen11sum,120CAR3,120CAR5,120CAR7,120CAR11
0,20 Microns Ltd.,2008-10-06,16.82,NaN,0.069713,-6.381449,-6.311735,-0.373052,-0.566450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20 Microns Ltd.,2008-10-07,15.05,-0.105232,0.023232,-0.669144,-0.645911,-1.502487,0.184699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.116178,-10.583954,-3.656212,-0.308820,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20 Microns Ltd.,2008-10-08,13.25,-0.119601,0.023232,-3.533362,-3.510130,-1.780674,0.072932,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.349361,0.091985,-11.254830,-3.066034,0.612259,NaN,0.228562,-12.593540,-5.876183,0.191663,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20 Microns Ltd.,2008-10-10,11.60,-0.124528,0.045520,-7.052324,-7.006804,0.217126,0.354629,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.182061,0.135616,-5.542948,-4.000644,0.573414,-0.388754,0.181132,-5.036699,-4.725787,1.836919,NaN,0.273128,-16.594178,-5.339991,1.320939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20 Microns Ltd.,2008-10-13,12.32,0.062069,0.066863,5.042738,5.109602,-2.437097,0.145853,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.163920,0.134667,-0.834193,-1.442626,1.579289,-0.183251,0.180182,-9.543586,-3.464452,1.702691,-0.163688,0.225697,-12.320480,-5.095004,1.701136,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13465088,Zylog Systems Ltd.,2024-03-21,0.35,0.000000,0.018215,1.441807,1.460021,0.572866,1.270303,-0.026702,0.003907,0.003905,-0.001484,0.049581,0.025001,0.115362,0.000000,0.054643,2.222872,0.647351,1.691011,0.000000,0.145734,0.669402,-0.213495,1.701467,0.000000,0.218599,1.086124,-0.832943,1.912107,NaN,NaN,NaN,NaN,NaN,0.016759,-0.011482,-0.029841,NaN
13465089,Zylog Systems Ltd.,2024-03-22,0.35,0.000000,0.018215,0.593324,0.611538,0.752002,0.124527,-0.026628,0.004073,0.003156,-0.001910,0.044392,0.019678,0.151766,0.000000,0.109307,1.990617,0.233272,1.835435,0.000000,0.145735,2.505059,-0.609317,1.943111,0.000000,0.182163,1.699158,-0.957902,1.419960,NaN,NaN,NaN,NaN,NaN,-0.034759,-0.017161,0.003050,NaN
13465090,Zylog Systems Ltd.,2024-03-26,0.35,0.000000,0.072878,-0.044514,0.028364,-1.091597,0.440605,-0.026633,0.004079,0.003177,-0.001971,0.045188,0.020494,0.145564,0.000000,0.109307,0.875511,-0.504666,0.376627,0.000000,0.145736,3.020372,-0.511135,1.553929,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.030633,-0.020203,NaN,NaN
13465091,Zylog Systems Ltd.,2024-03-27,0.35,0.000000,0.018215,0.326702,0.344916,-0.165072,-0.188505,-0.026979,0.004395,0.004305,-0.001647,0.062475,0.038228,0.057167,0.000000,0.109307,0.985242,-1.836003,0.159099,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.024533,NaN,NaN,NaN


In [16]:
output120CAR_2 = output120CAR.dropna(subset = "120CAR3").sort_values( by = ["CompanyName", "AsOnDate"] ).reset_index(drop = True)

In [17]:
output120CAR_2

,CompanyName,AsOnDate,ACP,pct,RF,RMRF,MF,SMB,HML,OLS120_intercept,OLS120_RMRF,OLS120_SMB,OLS120_HML,OLS120_r_squared,OLS120_adjusted_r_squared,OLS120_f_p_value,pct_cen3sum,RF_cen3sum,RMRF_cen3sum,SMB_cen3sum,HML_cen3sum,pct_cen5sum,RF_cen5sum,RMRF_cen5sum,SMB_cen5sum,HML_cen5sum,pct_cen7sum,RF_cen7sum,RMRF_cen7sum,SMB_cen7sum,HML_cen7sum,pct_cen11sum,RF_cen11sum,RMRF_cen11sum,SMB_cen11sum,HML_cen11sum,120CAR3,120CAR5,120CAR7,120CAR11
0,20 Microns Ltd.,2009-04-27,10.85,-0.011840,0.027124,-0.703543,-0.676419,0.213648,0.770117,-0.024856,0.012905,0.011293,0.001436,0.274365,0.255435,4.519505e-08,0.187444,0.045205,-2.338197,0.421393,1.511361,0.041191,0.064365,2.319174,-2.316852,1.941291,0.100704,0.119262,5.106493,-4.103135,1.985429,0.292514,0.178041,5.446112,-1.576223,1.119717,0.240052,0.094556,0.133024,0.333801
1,20 Microns Ltd.,2009-04-28,12.57,0.158525,0.009040,-3.021266,-3.012226,-0.063645,0.498647,-0.024314,0.013002,0.011581,0.002395,0.275674,0.256942,3.488935e-08,0.085429,0.045097,-1.235830,-1.015065,1.743395,0.085509,0.098807,4.071745,-2.282992,2.582601,-0.007437,0.117967,7.081665,-3.326557,2.449029,0.283085,0.156287,6.817385,-2.774410,1.315715,0.136922,0.075586,-0.014620,0.334596
2,20 Microns Ltd.,2009-04-29,11.80,-0.061257,0.008932,2.488979,2.497912,-1.165069,0.474631,-0.023559,0.012881,0.011145,0.003962,0.282966,0.264422,1.964126e-08,0.056590,0.062643,3.388675,-2.768029,1.569887,0.036800,0.098699,3.526662,-2.024770,2.251132,0.061530,0.116672,4.095830,-0.812131,2.565917,0.065935,0.154559,5.977789,-2.257276,1.954989,0.045605,0.024116,0.055896,0.110934
3,20 Microns Ltd.,2009-05-04,11.32,-0.040678,0.044670,3.920962,3.965632,-1.539316,0.596610,-0.024365,0.011572,0.008292,0.005490,0.248402,0.228964,2.834241e-07,-0.109885,0.062535,7.251471,-2.174774,0.982368,0.032611,0.080507,3.412760,-1.297168,1.553203,0.025297,0.116564,4.551217,-1.721034,2.055157,-0.098300,0.169834,5.359901,-1.725263,2.530758,-0.170596,0.036668,0.029613,-0.061727
4,20 Microns Ltd.,2009-05-05,11.23,-0.007951,0.008932,0.841529,0.850462,0.529611,-0.088872,-0.023842,0.010647,0.007331,0.007090,0.218311,0.198095,2.600825e-06,-0.064657,0.062535,3.945047,-0.068455,0.579926,-0.121389,0.080399,8.276026,-1.871037,0.786393,0.021821,0.097940,4.259972,-0.922170,1.539447,0.006246,0.168106,5.444229,-1.338551,2.588467,-0.101277,-0.162550,0.041268,0.033901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12986007,Zylog Systems Ltd.,2024-03-20,0.35,0.000000,0.018213,0.187741,0.205954,-0.677517,0.296181,-0.026576,0.003888,0.003108,-0.002064,0.043184,0.018438,1.616497e-01,0.000000,0.054642,0.120592,0.126100,1.136335,0.000000,0.127506,0.803937,0.423725,1.660006,0.000000,0.218598,0.390646,0.786768,1.386466,0.000000,0.291507,0.039579,-0.033743,0.057616,0.026571,0.004357,-0.033668,0.000899
12986008,Zylog Systems Ltd.,2024-03-21,0.35,0.000000,0.018215,1.441807,1.460021,0.572866,1.270303,-0.026702,0.003907,0.003905,-0.001484,0.049581,0.025001,1.153618e-01,0.000000,0.054643,2.222872,0.647351,1.691011,0.000000,0.145734,0.669402,-0.213495,1.701467,0.000000,0.218599,1.086124,-0.832943,1.912107,NaN,NaN,NaN,NaN,NaN,0.016759,-0.011482,-0.029841,NaN
12986009,Zylog Systems Ltd.,2024-03-22,0.35,0.000000,0.018215,0.593324,0.611538,0.752002,0.124527,-0.026628,0.004073,0.003156,-0.001910,0.044392,0.019678,1.517656e-01,0.000000,0.109307,1.990617,0.233272,1.835435,0.000000,0.145735,2.505059,-0.609317,1.943111,0.000000,0.182163,1.699158,-0.957902,1.419960,NaN,NaN,NaN,NaN,NaN,-0.034759,-0.017161,0.003050,NaN
12986010,Zylog Systems Ltd.,2024-03-26,0.35,0.000000,0.072878,-0.044514,0.028364,-1.091597,0.440605,-0.026633,0.004079,0.003177,-0.001971,0.045188,0.020494,1.455639e-01,0.000000,0.109307,0.875511,-0.504666,0.376627,0.000000,0.145736,3.020372,-0.511135,1.553929,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.030633,-0.020203,NaN,NaN


In [18]:
output120CAR_2.to_csv("output120CAR.csv")

## Data Reading

In [2]:
ols150 = pd.read_csv("ols150.csv", index_col = 0)

In [3]:
ols150["AsOnDate"] = pd.to_datetime(ols150["AsOnDate"], format = "%Y-%m-%d")

## 150 CAR Calc

In [4]:
ols150CAR = ols150.sort_values(by = ["CompanyName", "AsOnDate"])
ols150CAR

,CompanyName,AsOnDate,ACP,pct,RF,RMRF,MF,SMB,HML,OLS150_intercept,OLS150_RMRF,OLS150_SMB,OLS150_HML,OLS150_r_squared,OLS150_adjusted_r_squared,OLS150_f_p_value
0,20 Microns Ltd.,2008-10-06,16.82,NaN,0.069713,-6.381449,-6.311735,-0.373052,-0.566450,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20 Microns Ltd.,2008-10-07,15.05,-0.105232,0.023232,-0.669144,-0.645911,-1.502487,0.184699,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20 Microns Ltd.,2008-10-08,13.25,-0.119601,0.023232,-3.533362,-3.510130,-1.780674,0.072932,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20 Microns Ltd.,2008-10-10,11.60,-0.124528,0.045520,-7.052324,-7.006804,0.217126,0.354629,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20 Microns Ltd.,2008-10-13,12.32,0.062069,0.066863,5.042738,5.109602,-2.437097,0.145853,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13465088,Zylog Systems Ltd.,2024-03-21,0.35,0.000000,0.018215,1.441807,1.460021,0.572866,1.270303,-0.026475,0.002737,0.003711,-0.001810,0.036499,0.016701,0.141841
13465089,Zylog Systems Ltd.,2024-03-22,0.35,0.000000,0.018215,0.593324,0.611538,0.752002,0.124527,-0.026415,0.002826,0.003197,-0.002014,0.032462,0.012581,0.184321
13465090,Zylog Systems Ltd.,2024-03-26,0.35,0.000000,0.072878,-0.044514,0.028364,-1.091597,0.440605,-0.026460,0.002981,0.003214,-0.002013,0.033903,0.014052,0.167967
13465091,Zylog Systems Ltd.,2024-03-27,0.35,0.000000,0.018215,0.326702,0.344916,-0.165072,-0.188505,-0.026802,0.003385,0.004068,-0.001858,0.047152,0.027573,0.069520


In [5]:
# FUNCTION

# def CAR150(frame):
#     if ((~frame.OLS150_intercept.isnull().any()) == True):
#         ER = frame.RF + (frame.RMRF * frame.OLS150_RMRF) + (frame.SMB * frame.OLS150_SMB) + (frame.HML * frame.OLS150_HML) + frame.OLS150_intercept
#         AR = frame.pct - ER
#         return AR.sum()
#     else: 
#         return np.NaN

    # for i in range(len(frame)):
    #     if (i>0) & ( i< len(frame)-1) :
    #         coeff150CAR = np.array([frame.iloc[i]["OLS150_RMRF"], frame.iloc[i]["OLS150_SMB"],
    #                                 frame.iloc[i]["OLS150_HML"], 1, frame.iloc[i]["OLS150_intercept"]])

    #         event150CAR3 = np.array(frame.loc[ [i-1, i, i+1], ["RMRF", "SMB", "HML", "RF"]],)
    #         event150CAR3 = [np.append(item, 1) for item in event150CAR3]

    #         ER = np.matmul(event150CAR3, coeff150CAR)

    #         actualReturns = np.array([frame.iloc[i-1]["pct"], frame.iloc[i]["pct"], frame.iloc[i+1]["pct"]])

    #         AR = np.subtract( actualReturns, ER )

    #         output150CAR3.loc[i, ["150CAR3"]] = AR.sum()


    
        # if i>=2:
        #     frame["150CAR5"].iloc[i] = CAR150(frame.iloc[ i-2 : i+2 ])
        # if i>=3:
        #     frame["150CAR7"].iloc[i] = CAR150(frame.iloc[ i-3 : i+3 ])
        # if i>=5:
        #     frame["150CAR11"].iloc[i] = CAR150(frame.iloc[ i-5 : i+5 ])

        

def CAR(frame):

    outputFrame = pd.DataFrame( index = frame.index)

        
    # 3 Days
    
    outputFrame["pct_cen3sum"] = frame["pct"].rolling(window = 3, min_periods = 3, center = True).sum()
    
    outputFrame["RF_cen3sum"] = frame["RF"].rolling(window = 3, min_periods = 3, center = True).sum()
    
    outputFrame["RMRF_cen3sum"] = frame["RMRF"].rolling(window = 3, min_periods = 3, center = True).sum()
    outputFrame["SMB_cen3sum"] = frame["SMB"].rolling(window = 3, min_periods = 3, center = True).sum()
    outputFrame["HML_cen3sum"] = frame["HML"].rolling(window = 3, min_periods = 3, center = True).sum()
    
    
    # 5 Days
    
    outputFrame["pct_cen5sum"] = frame["pct"].rolling(window = 5, min_periods = 5, center = True).sum()
    
    outputFrame["RF_cen5sum"] = frame["RF"].rolling(window = 5, min_periods = 5, center = True).sum()
    
    outputFrame["RMRF_cen5sum"] = frame["RMRF"].rolling(window = 5, min_periods = 5, center = True).sum()
    outputFrame["SMB_cen5sum"] = frame["SMB"].rolling(window = 5, min_periods = 5, center = True).sum()
    outputFrame["HML_cen5sum"] = frame["HML"].rolling(window = 5, min_periods = 5, center = True).sum()
    
    
    # 7 Days
    
    outputFrame["pct_cen7sum"] = frame["pct"].rolling(window = 7, min_periods = 7, center = True).sum()
    
    outputFrame["RF_cen7sum"] = frame["RF"].rolling(window = 7, min_periods = 7, center = True).sum()
    
    outputFrame["RMRF_cen7sum"] = frame["RMRF"].rolling(window = 7, min_periods = 7, center = True).sum()
    outputFrame["SMB_cen7sum"] = frame["SMB"].rolling(window = 7, min_periods = 7, center = True).sum()
    outputFrame["HML_cen7sum"] = frame["HML"].rolling(window = 7, min_periods = 7, center = True).sum()
    
    
    # 11 Days
    
    outputFrame["pct_cen11sum"] = frame["pct"].rolling(window = 11, min_periods = 11, center = True).sum()
    
    outputFrame["RF_cen11sum"] = frame["RF"].rolling(window = 11, min_periods = 11, center = True).sum()
    
    outputFrame["RMRF_cen11sum"] = frame["RMRF"].rolling(window = 11, min_periods = 11, center = True).sum()
    outputFrame["SMB_cen11sum"] = frame["SMB"].rolling(window = 11, min_periods = 11, center = True).sum()
    outputFrame["HML_cen11sum"] = frame["HML"].rolling(window = 11, min_periods = 11, center = True).sum()
    
    
    return outputFrame


In [6]:
result150CAR = ols150CAR.sort_values(by = ["CompanyName", "AsOnDate"]).groupby(by = "CompanyName").progress_apply(CAR)

  0%|          | 0/3721 [00:00<?, ?it/s]

In [7]:
result150CAR2 = result150CAR.reset_index(level = 0).drop("CompanyName", axis = 1)
result150CAR2

,pct_cen3sum,RF_cen3sum,RMRF_cen3sum,SMB_cen3sum,HML_cen3sum,pct_cen5sum,RF_cen5sum,RMRF_cen5sum,SMB_cen5sum,HML_cen5sum,pct_cen7sum,RF_cen7sum,RMRF_cen7sum,SMB_cen7sum,HML_cen7sum,pct_cen11sum,RF_cen11sum,RMRF_cen11sum,SMB_cen11sum,HML_cen11sum
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,0.116178,-10.583954,-3.656212,-0.308820,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.349361,0.091985,-11.254830,-3.066034,0.612259,NaN,0.228562,-12.593540,-5.876183,0.191663,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-0.182061,0.135616,-5.542948,-4.000644,0.573414,-0.388754,0.181132,-5.036699,-4.725787,1.836919,NaN,0.273128,-16.594178,-5.339991,1.320939,NaN,NaN,NaN,NaN,NaN
4,-0.163920,0.134667,-0.834193,-1.442626,1.579289,-0.183251,0.180182,-9.543586,-3.464452,1.702691,-0.163688,0.225697,-12.320480,-5.095004,1.701136,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13465088,0.000000,0.054643,2.222872,0.647351,1.691011,0.000000,0.145734,0.669402,-0.213495,1.701467,0.000000,0.218599,1.086124,-0.832943,1.912107,NaN,NaN,NaN,NaN,NaN
13465089,0.000000,0.109307,1.990617,0.233272,1.835435,0.000000,0.145735,2.505059,-0.609317,1.943111,0.000000,0.182163,1.699158,-0.957902,1.419960,NaN,NaN,NaN,NaN,NaN
13465090,0.000000,0.109307,0.875511,-0.504666,0.376627,0.000000,0.145736,3.020372,-0.511135,1.553929,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13465091,0.000000,0.109307,0.985242,-1.836003,0.159099,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
output150CAR = pd.concat([ols150CAR, result150CAR2], axis = 1)
output150CAR

,CompanyName,AsOnDate,ACP,pct,RF,RMRF,MF,SMB,HML,OLS150_intercept,OLS150_RMRF,OLS150_SMB,OLS150_HML,OLS150_r_squared,OLS150_adjusted_r_squared,OLS150_f_p_value,pct_cen3sum,RF_cen3sum,RMRF_cen3sum,SMB_cen3sum,HML_cen3sum,pct_cen5sum,RF_cen5sum,RMRF_cen5sum,SMB_cen5sum,HML_cen5sum,pct_cen7sum,RF_cen7sum,RMRF_cen7sum,SMB_cen7sum,HML_cen7sum,pct_cen11sum,RF_cen11sum,RMRF_cen11sum,SMB_cen11sum,HML_cen11sum
0,20 Microns Ltd.,2008-10-06,16.82,NaN,0.069713,-6.381449,-6.311735,-0.373052,-0.566450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20 Microns Ltd.,2008-10-07,15.05,-0.105232,0.023232,-0.669144,-0.645911,-1.502487,0.184699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.116178,-10.583954,-3.656212,-0.308820,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20 Microns Ltd.,2008-10-08,13.25,-0.119601,0.023232,-3.533362,-3.510130,-1.780674,0.072932,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.349361,0.091985,-11.254830,-3.066034,0.612259,NaN,0.228562,-12.593540,-5.876183,0.191663,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20 Microns Ltd.,2008-10-10,11.60,-0.124528,0.045520,-7.052324,-7.006804,0.217126,0.354629,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.182061,0.135616,-5.542948,-4.000644,0.573414,-0.388754,0.181132,-5.036699,-4.725787,1.836919,NaN,0.273128,-16.594178,-5.339991,1.320939,NaN,NaN,NaN,NaN,NaN
4,20 Microns Ltd.,2008-10-13,12.32,0.062069,0.066863,5.042738,5.109602,-2.437097,0.145853,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.163920,0.134667,-0.834193,-1.442626,1.579289,-0.183251,0.180182,-9.543586,-3.464452,1.702691,-0.163688,0.225697,-12.320480,-5.095004,1.701136,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13465088,Zylog Systems Ltd.,2024-03-21,0.35,0.000000,0.018215,1.441807,1.460021,0.572866,1.270303,-0.026475,0.002737,0.003711,-0.001810,0.036499,0.016701,0.141841,0.000000,0.054643,2.222872,0.647351,1.691011,0.000000,0.145734,0.669402,-0.213495,1.701467,0.000000,0.218599,1.086124,-0.832943,1.912107,NaN,NaN,NaN,NaN,NaN
13465089,Zylog Systems Ltd.,2024-03-22,0.35,0.000000,0.018215,0.593324,0.611538,0.752002,0.124527,-0.026415,0.002826,0.003197,-0.002014,0.032462,0.012581,0.184321,0.000000,0.109307,1.990617,0.233272,1.835435,0.000000,0.145735,2.505059,-0.609317,1.943111,0.000000,0.182163,1.699158,-0.957902,1.419960,NaN,NaN,NaN,NaN,NaN
13465090,Zylog Systems Ltd.,2024-03-26,0.35,0.000000,0.072878,-0.044514,0.028364,-1.091597,0.440605,-0.026460,0.002981,0.003214,-0.002013,0.033903,0.014052,0.167967,0.000000,0.109307,0.875511,-0.504666,0.376627,0.000000,0.145736,3.020372,-0.511135,1.553929,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13465091,Zylog Systems Ltd.,2024-03-27,0.35,0.000000,0.018215,0.326702,0.344916,-0.165072,-0.188505,-0.026802,0.003385,0.004068,-0.001858,0.047152,0.027573,0.069520,0.000000,0.109307,0.985242,-1.836003,0.159099,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
output150CAR.columns

Index(['CompanyName', 'AsOnDate', 'ACP', 'pct', 'RF', 'RMRF', 'MF', 'SMB',
       'HML', 'OLS150_intercept', 'OLS150_RMRF', 'OLS150_SMB', 'OLS150_HML',
       'OLS150_r_squared', 'OLS150_adjusted_r_squared', 'OLS150_f_p_value',
       'pct_cen3sum', 'RF_cen3sum', 'RMRF_cen3sum', 'SMB_cen3sum',
       'HML_cen3sum', 'pct_cen5sum', 'RF_cen5sum', 'RMRF_cen5sum',
       'SMB_cen5sum', 'HML_cen5sum', 'pct_cen7sum', 'RF_cen7sum',
       'RMRF_cen7sum', 'SMB_cen7sum', 'HML_cen7sum', 'pct_cen11sum',
       'RF_cen11sum', 'RMRF_cen11sum', 'SMB_cen11sum', 'HML_cen11sum'],
      dtype='object')

In [10]:
output150CAR["150CAR3"] = output150CAR["pct_cen3sum"] - (output150CAR["RF_cen3sum"] + 
                          
                          output150CAR["OLS150_RMRF"]*output150CAR["RMRF_cen3sum"] + 
                          output150CAR["OLS150_SMB"]*output150CAR["SMB_cen3sum"] + 
                          output150CAR["OLS150_HML"]*output150CAR["HML_cen3sum"] + 
                          output150CAR["OLS150_intercept"]*3 )

In [11]:
output150CAR["150CAR5"] = output150CAR["pct_cen5sum"] - (output150CAR["RF_cen5sum"] + 
                          
                          output150CAR["OLS150_RMRF"]*output150CAR["RMRF_cen5sum"] + 
                          output150CAR["OLS150_SMB"]*output150CAR["SMB_cen5sum"] + 
                          output150CAR["OLS150_HML"]*output150CAR["HML_cen5sum"] + 
                          output150CAR["OLS150_intercept"]*5 )

In [12]:
output150CAR["150CAR7"] = output150CAR["pct_cen7sum"] - (output150CAR["RF_cen7sum"] + 
                          
                          output150CAR["OLS150_RMRF"]*output150CAR["RMRF_cen7sum"] + 
                          output150CAR["OLS150_SMB"]*output150CAR["SMB_cen7sum"] + 
                          output150CAR["OLS150_HML"]*output150CAR["HML_cen7sum"] + 
                          output150CAR["OLS150_intercept"]*7 )

In [13]:
output150CAR["150CAR11"] = output150CAR["pct_cen11sum"] - (output150CAR["RF_cen11sum"] + 
                          
                          output150CAR["OLS150_RMRF"]*output150CAR["RMRF_cen11sum"] + 
                          output150CAR["OLS150_SMB"]*output150CAR["SMB_cen11sum"] + 
                          output150CAR["OLS150_HML"]*output150CAR["HML_cen11sum"] + 
                          output150CAR["OLS150_intercept"]*11 )

In [14]:
output150CAR

,CompanyName,AsOnDate,ACP,pct,RF,RMRF,MF,SMB,HML,OLS150_intercept,OLS150_RMRF,OLS150_SMB,OLS150_HML,OLS150_r_squared,OLS150_adjusted_r_squared,OLS150_f_p_value,pct_cen3sum,RF_cen3sum,RMRF_cen3sum,SMB_cen3sum,HML_cen3sum,pct_cen5sum,RF_cen5sum,RMRF_cen5sum,SMB_cen5sum,HML_cen5sum,pct_cen7sum,RF_cen7sum,RMRF_cen7sum,SMB_cen7sum,HML_cen7sum,pct_cen11sum,RF_cen11sum,RMRF_cen11sum,SMB_cen11sum,HML_cen11sum,150CAR3,150CAR5,150CAR7,150CAR11
0,20 Microns Ltd.,2008-10-06,16.82,NaN,0.069713,-6.381449,-6.311735,-0.373052,-0.566450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20 Microns Ltd.,2008-10-07,15.05,-0.105232,0.023232,-0.669144,-0.645911,-1.502487,0.184699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.116178,-10.583954,-3.656212,-0.308820,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20 Microns Ltd.,2008-10-08,13.25,-0.119601,0.023232,-3.533362,-3.510130,-1.780674,0.072932,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.349361,0.091985,-11.254830,-3.066034,0.612259,NaN,0.228562,-12.593540,-5.876183,0.191663,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20 Microns Ltd.,2008-10-10,11.60,-0.124528,0.045520,-7.052324,-7.006804,0.217126,0.354629,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.182061,0.135616,-5.542948,-4.000644,0.573414,-0.388754,0.181132,-5.036699,-4.725787,1.836919,NaN,0.273128,-16.594178,-5.339991,1.320939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20 Microns Ltd.,2008-10-13,12.32,0.062069,0.066863,5.042738,5.109602,-2.437097,0.145853,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.163920,0.134667,-0.834193,-1.442626,1.579289,-0.183251,0.180182,-9.543586,-3.464452,1.702691,-0.163688,0.225697,-12.320480,-5.095004,1.701136,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13465088,Zylog Systems Ltd.,2024-03-21,0.35,0.000000,0.018215,1.441807,1.460021,0.572866,1.270303,-0.026475,0.002737,0.003711,-0.001810,0.036499,0.016701,0.141841,0.000000,0.054643,2.222872,0.647351,1.691011,0.000000,0.145734,0.669402,-0.213495,1.701467,0.000000,0.218599,1.086124,-0.832943,1.912107,NaN,NaN,NaN,NaN,NaN,0.019359,-0.011318,-0.029693,NaN
13465089,Zylog Systems Ltd.,2024-03-22,0.35,0.000000,0.018215,0.593324,0.611538,0.752002,0.124527,-0.026415,0.002826,0.003197,-0.002014,0.032462,0.012581,0.184321,0.000000,0.109307,1.990617,0.233272,1.835435,0.000000,0.145735,2.505059,-0.609317,1.943111,0.000000,0.182163,1.699158,-0.957902,1.419960,NaN,NaN,NaN,NaN,NaN,-0.032738,-0.014879,0.003861,NaN
13465090,Zylog Systems Ltd.,2024-03-26,0.35,0.000000,0.072878,-0.044514,0.028364,-1.091597,0.440605,-0.026460,0.002981,0.003214,-0.002013,0.033903,0.014052,0.167967,0.000000,0.109307,0.875511,-0.504666,0.376627,0.000000,0.145736,3.020372,-0.511135,1.553929,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.030156,-0.017667,NaN,NaN
13465091,Zylog Systems Ltd.,2024-03-27,0.35,0.000000,0.018215,0.326702,0.344916,-0.165072,-0.188505,-0.026802,0.003385,0.004068,-0.001858,0.047152,0.027573,0.069520,0.000000,0.109307,0.985242,-1.836003,0.159099,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.024470,NaN,NaN,NaN


In [16]:
output150CAR_2 = output150CAR.dropna(subset = "150CAR3").sort_values( by = ["CompanyName", "AsOnDate"] ).reset_index(drop = True)

In [17]:
output150CAR_2.to_csv("output150CAR.csv")

## Data Reading

In [21]:
ols180 = pd.read_csv("ols180.csv", index_col = 0)

In [22]:
ols180["AsOnDate"] = pd.to_datetime(ols180["AsOnDate"], format = "%Y-%m-%d")

## 180 CAR Calc

In [23]:
ols180CAR = ols180.sort_values(by = ["CompanyName", "AsOnDate"])
ols180CAR

,CompanyName,AsOnDate,ACP,pct,RF,RMRF,MF,SMB,HML,OLS180_intercept,OLS180_RMRF,OLS180_SMB,OLS180_HML,OLS180_r_squared,OLS180_adjusted_r_squared,OLS180_f_p_value
0,20 Microns Ltd.,2008-10-06,16.82,NaN,0.069713,-6.381449,-6.311735,-0.373052,-0.566450,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20 Microns Ltd.,2008-10-07,15.05,-0.105232,0.023232,-0.669144,-0.645911,-1.502487,0.184699,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20 Microns Ltd.,2008-10-08,13.25,-0.119601,0.023232,-3.533362,-3.510130,-1.780674,0.072932,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20 Microns Ltd.,2008-10-10,11.60,-0.124528,0.045520,-7.052324,-7.006804,0.217126,0.354629,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20 Microns Ltd.,2008-10-13,12.32,0.062069,0.066863,5.042738,5.109602,-2.437097,0.145853,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13465088,Zylog Systems Ltd.,2024-03-21,0.35,0.000000,0.018215,1.441807,1.460021,0.572866,1.270303,-0.026511,0.002663,0.002803,-0.002113,0.025498,0.008887,0.207118
13465089,Zylog Systems Ltd.,2024-03-22,0.35,0.000000,0.018215,0.593324,0.611538,0.752002,0.124527,-0.026262,0.002406,0.002773,-0.002086,0.024682,0.008058,0.220363
13465090,Zylog Systems Ltd.,2024-03-26,0.35,0.000000,0.072878,-0.044514,0.028364,-1.091597,0.440605,-0.026266,0.002386,0.002795,-0.002064,0.024686,0.008062,0.220298
13465091,Zylog Systems Ltd.,2024-03-27,0.35,0.000000,0.018215,0.326702,0.344916,-0.165072,-0.188505,-0.026598,0.002727,0.003744,-0.001882,0.035403,0.018961,0.095276


In [24]:
# FUNCTION

# def CAR180(frame):
#     if ((~frame.OLS180_intercept.isnull().any()) == True):
#         ER = frame.RF + (frame.RMRF * frame.OLS180_RMRF) + (frame.SMB * frame.OLS180_SMB) + (frame.HML * frame.OLS180_HML) + frame.OLS180_intercept
#         AR = frame.pct - ER
#         return AR.sum()
#     else: 
#         return np.NaN

    # for i in range(len(frame)):
    #     if (i>0) & ( i< len(frame)-1) :
    #         coeff180CAR = np.array([frame.iloc[i]["OLS180_RMRF"], frame.iloc[i]["OLS180_SMB"],
    #                                 frame.iloc[i]["OLS180_HML"], 1, frame.iloc[i]["OLS180_intercept"]])

    #         event180CAR3 = np.array(frame.loc[ [i-1, i, i+1], ["RMRF", "SMB", "HML", "RF"]],)
    #         event180CAR3 = [np.append(item, 1) for item in event180CAR3]

    #         ER = np.matmul(event180CAR3, coeff180CAR)

    #         actualReturns = np.array([frame.iloc[i-1]["pct"], frame.iloc[i]["pct"], frame.iloc[i+1]["pct"]])

    #         AR = np.subtract( actualReturns, ER )

    #         output180CAR3.loc[i, ["180CAR3"]] = AR.sum()


    
        # if i>=2:
        #     frame["180CAR5"].iloc[i] = CAR180(frame.iloc[ i-2 : i+2 ])
        # if i>=3:
        #     frame["180CAR7"].iloc[i] = CAR180(frame.iloc[ i-3 : i+3 ])
        # if i>=5:
        #     frame["180CAR11"].iloc[i] = CAR180(frame.iloc[ i-5 : i+5 ])

        

def CAR(frame):

    outputFrame = pd.DataFrame( index = frame.index)

        
    # 3 Days
    
    outputFrame["pct_cen3sum"] = frame["pct"].rolling(window = 3, min_periods = 3, center = True).sum()
    
    outputFrame["RF_cen3sum"] = frame["RF"].rolling(window = 3, min_periods = 3, center = True).sum()
    
    outputFrame["RMRF_cen3sum"] = frame["RMRF"].rolling(window = 3, min_periods = 3, center = True).sum()
    outputFrame["SMB_cen3sum"] = frame["SMB"].rolling(window = 3, min_periods = 3, center = True).sum()
    outputFrame["HML_cen3sum"] = frame["HML"].rolling(window = 3, min_periods = 3, center = True).sum()
    
    
    # 5 Days
    
    outputFrame["pct_cen5sum"] = frame["pct"].rolling(window = 5, min_periods = 5, center = True).sum()
    
    outputFrame["RF_cen5sum"] = frame["RF"].rolling(window = 5, min_periods = 5, center = True).sum()
    
    outputFrame["RMRF_cen5sum"] = frame["RMRF"].rolling(window = 5, min_periods = 5, center = True).sum()
    outputFrame["SMB_cen5sum"] = frame["SMB"].rolling(window = 5, min_periods = 5, center = True).sum()
    outputFrame["HML_cen5sum"] = frame["HML"].rolling(window = 5, min_periods = 5, center = True).sum()
    
    
    # 7 Days
    
    outputFrame["pct_cen7sum"] = frame["pct"].rolling(window = 7, min_periods = 7, center = True).sum()
    
    outputFrame["RF_cen7sum"] = frame["RF"].rolling(window = 7, min_periods = 7, center = True).sum()
    
    outputFrame["RMRF_cen7sum"] = frame["RMRF"].rolling(window = 7, min_periods = 7, center = True).sum()
    outputFrame["SMB_cen7sum"] = frame["SMB"].rolling(window = 7, min_periods = 7, center = True).sum()
    outputFrame["HML_cen7sum"] = frame["HML"].rolling(window = 7, min_periods = 7, center = True).sum()
    
    
    # 11 Days
    
    outputFrame["pct_cen11sum"] = frame["pct"].rolling(window = 11, min_periods = 11, center = True).sum()
    
    outputFrame["RF_cen11sum"] = frame["RF"].rolling(window = 11, min_periods = 11, center = True).sum()
    
    outputFrame["RMRF_cen11sum"] = frame["RMRF"].rolling(window = 11, min_periods = 11, center = True).sum()
    outputFrame["SMB_cen11sum"] = frame["SMB"].rolling(window = 11, min_periods = 11, center = True).sum()
    outputFrame["HML_cen11sum"] = frame["HML"].rolling(window = 11, min_periods = 11, center = True).sum()
    
    
    return outputFrame


In [25]:
result180CAR = ols180CAR.sort_values(by = ["CompanyName", "AsOnDate"]).groupby(by = "CompanyName").progress_apply(CAR)

  0%|          | 0/3721 [00:00<?, ?it/s]

In [26]:
result180CAR2 = result180CAR.reset_index(level = 0).drop("CompanyName", axis = 1)
result180CAR2

,pct_cen3sum,RF_cen3sum,RMRF_cen3sum,SMB_cen3sum,HML_cen3sum,pct_cen5sum,RF_cen5sum,RMRF_cen5sum,SMB_cen5sum,HML_cen5sum,pct_cen7sum,RF_cen7sum,RMRF_cen7sum,SMB_cen7sum,HML_cen7sum,pct_cen11sum,RF_cen11sum,RMRF_cen11sum,SMB_cen11sum,HML_cen11sum
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,0.116178,-10.583954,-3.656212,-0.308820,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.349361,0.091985,-11.254830,-3.066034,0.612259,NaN,0.228562,-12.593540,-5.876183,0.191663,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-0.182061,0.135616,-5.542948,-4.000644,0.573414,-0.388754,0.181132,-5.036699,-4.725787,1.836919,NaN,0.273128,-16.594178,-5.339991,1.320939,NaN,NaN,NaN,NaN,NaN
4,-0.163920,0.134667,-0.834193,-1.442626,1.579289,-0.183251,0.180182,-9.543586,-3.464452,1.702691,-0.163688,0.225697,-12.320480,-5.095004,1.701136,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13465088,0.000000,0.054643,2.222872,0.647351,1.691011,0.000000,0.145734,0.669402,-0.213495,1.701467,0.000000,0.218599,1.086124,-0.832943,1.912107,NaN,NaN,NaN,NaN,NaN
13465089,0.000000,0.109307,1.990617,0.233272,1.835435,0.000000,0.145735,2.505059,-0.609317,1.943111,0.000000,0.182163,1.699158,-0.957902,1.419960,NaN,NaN,NaN,NaN,NaN
13465090,0.000000,0.109307,0.875511,-0.504666,0.376627,0.000000,0.145736,3.020372,-0.511135,1.553929,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13465091,0.000000,0.109307,0.985242,-1.836003,0.159099,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
output180CAR = pd.concat([ols180CAR, result180CAR2], axis = 1)
output180CAR

,CompanyName,AsOnDate,ACP,pct,RF,RMRF,MF,SMB,HML,OLS180_intercept,OLS180_RMRF,OLS180_SMB,OLS180_HML,OLS180_r_squared,OLS180_adjusted_r_squared,OLS180_f_p_value,pct_cen3sum,RF_cen3sum,RMRF_cen3sum,SMB_cen3sum,HML_cen3sum,pct_cen5sum,RF_cen5sum,RMRF_cen5sum,SMB_cen5sum,HML_cen5sum,pct_cen7sum,RF_cen7sum,RMRF_cen7sum,SMB_cen7sum,HML_cen7sum,pct_cen11sum,RF_cen11sum,RMRF_cen11sum,SMB_cen11sum,HML_cen11sum
0,20 Microns Ltd.,2008-10-06,16.82,NaN,0.069713,-6.381449,-6.311735,-0.373052,-0.566450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20 Microns Ltd.,2008-10-07,15.05,-0.105232,0.023232,-0.669144,-0.645911,-1.502487,0.184699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.116178,-10.583954,-3.656212,-0.308820,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20 Microns Ltd.,2008-10-08,13.25,-0.119601,0.023232,-3.533362,-3.510130,-1.780674,0.072932,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.349361,0.091985,-11.254830,-3.066034,0.612259,NaN,0.228562,-12.593540,-5.876183,0.191663,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20 Microns Ltd.,2008-10-10,11.60,-0.124528,0.045520,-7.052324,-7.006804,0.217126,0.354629,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.182061,0.135616,-5.542948,-4.000644,0.573414,-0.388754,0.181132,-5.036699,-4.725787,1.836919,NaN,0.273128,-16.594178,-5.339991,1.320939,NaN,NaN,NaN,NaN,NaN
4,20 Microns Ltd.,2008-10-13,12.32,0.062069,0.066863,5.042738,5.109602,-2.437097,0.145853,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.163920,0.134667,-0.834193,-1.442626,1.579289,-0.183251,0.180182,-9.543586,-3.464452,1.702691,-0.163688,0.225697,-12.320480,-5.095004,1.701136,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13465088,Zylog Systems Ltd.,2024-03-21,0.35,0.000000,0.018215,1.441807,1.460021,0.572866,1.270303,-0.026511,0.002663,0.002803,-0.002113,0.025498,0.008887,0.207118,0.000000,0.054643,2.222872,0.647351,1.691011,0.000000,0.145734,0.669402,-0.213495,1.701467,0.000000,0.218599,1.086124,-0.832943,1.912107,NaN,NaN,NaN,NaN,NaN
13465089,Zylog Systems Ltd.,2024-03-22,0.35,0.000000,0.018215,0.593324,0.611538,0.752002,0.124527,-0.026262,0.002406,0.002773,-0.002086,0.024682,0.008058,0.220363,0.000000,0.109307,1.990617,0.233272,1.835435,0.000000,0.145735,2.505059,-0.609317,1.943111,0.000000,0.182163,1.699158,-0.957902,1.419960,NaN,NaN,NaN,NaN,NaN
13465090,Zylog Systems Ltd.,2024-03-26,0.35,0.000000,0.072878,-0.044514,0.028364,-1.091597,0.440605,-0.026266,0.002386,0.002795,-0.002064,0.024686,0.008062,0.220298,0.000000,0.109307,0.875511,-0.504666,0.376627,0.000000,0.145736,3.020372,-0.511135,1.553929,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13465091,Zylog Systems Ltd.,2024-03-27,0.35,0.000000,0.018215,0.326702,0.344916,-0.165072,-0.188505,-0.026598,0.002727,0.003744,-0.001882,0.035403,0.018961,0.095276,0.000000,0.109307,0.985242,-1.836003,0.159099,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
output180CAR.columns

Index(['CompanyName', 'AsOnDate', 'ACP', 'pct', 'RF', 'RMRF', 'MF', 'SMB',
       'HML', 'OLS180_intercept', 'OLS180_RMRF', 'OLS180_SMB', 'OLS180_HML',
       'OLS180_r_squared', 'OLS180_adjusted_r_squared', 'OLS180_f_p_value',
       'pct_cen3sum', 'RF_cen3sum', 'RMRF_cen3sum', 'SMB_cen3sum',
       'HML_cen3sum', 'pct_cen5sum', 'RF_cen5sum', 'RMRF_cen5sum',
       'SMB_cen5sum', 'HML_cen5sum', 'pct_cen7sum', 'RF_cen7sum',
       'RMRF_cen7sum', 'SMB_cen7sum', 'HML_cen7sum', 'pct_cen11sum',
       'RF_cen11sum', 'RMRF_cen11sum', 'SMB_cen11sum', 'HML_cen11sum'],
      dtype='object')

In [29]:
output180CAR["180CAR3"] = output180CAR["pct_cen3sum"] - (output180CAR["RF_cen3sum"] + 
                          
                          output180CAR["OLS180_RMRF"]*output180CAR["RMRF_cen3sum"] + 
                          output180CAR["OLS180_SMB"]*output180CAR["SMB_cen3sum"] + 
                          output180CAR["OLS180_HML"]*output180CAR["HML_cen3sum"] + 
                          output180CAR["OLS180_intercept"]*3 )


In [30]:
output180CAR["180CAR5"] = output180CAR["pct_cen5sum"] - (output180CAR["RF_cen5sum"] + 
                          
                          output180CAR["OLS180_RMRF"]*output180CAR["RMRF_cen5sum"] + 
                          output180CAR["OLS180_SMB"]*output180CAR["SMB_cen5sum"] + 
                          output180CAR["OLS180_HML"]*output180CAR["HML_cen5sum"] + 
                          output180CAR["OLS180_intercept"]*5 )

In [31]:
output180CAR["180CAR7"] = output180CAR["pct_cen7sum"] - (output180CAR["RF_cen7sum"] + 
                          
                          output180CAR["OLS180_RMRF"]*output180CAR["RMRF_cen7sum"] + 
                          output180CAR["OLS180_SMB"]*output180CAR["SMB_cen7sum"] + 
                          output180CAR["OLS180_HML"]*output180CAR["HML_cen7sum"] + 
                          output180CAR["OLS180_intercept"]*7 )

In [32]:
output180CAR["180CAR11"] = output180CAR["pct_cen11sum"] - (output180CAR["RF_cen11sum"] + 
                          
                          output180CAR["OLS180_RMRF"]*output180CAR["RMRF_cen11sum"] + 
                          output180CAR["OLS180_SMB"]*output180CAR["SMB_cen11sum"] + 
                          output180CAR["OLS180_HML"]*output180CAR["HML_cen11sum"] + 
                          output180CAR["OLS180_intercept"]*11 )

In [33]:
output180CAR

,CompanyName,AsOnDate,ACP,pct,RF,RMRF,MF,SMB,HML,OLS180_intercept,OLS180_RMRF,OLS180_SMB,OLS180_HML,OLS180_r_squared,OLS180_adjusted_r_squared,OLS180_f_p_value,pct_cen3sum,RF_cen3sum,RMRF_cen3sum,SMB_cen3sum,HML_cen3sum,pct_cen5sum,RF_cen5sum,RMRF_cen5sum,SMB_cen5sum,HML_cen5sum,pct_cen7sum,RF_cen7sum,RMRF_cen7sum,SMB_cen7sum,HML_cen7sum,pct_cen11sum,RF_cen11sum,RMRF_cen11sum,SMB_cen11sum,HML_cen11sum,180CAR3,180CAR5,180CAR7,180CAR11
0,20 Microns Ltd.,2008-10-06,16.82,NaN,0.069713,-6.381449,-6.311735,-0.373052,-0.566450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20 Microns Ltd.,2008-10-07,15.05,-0.105232,0.023232,-0.669144,-0.645911,-1.502487,0.184699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.116178,-10.583954,-3.656212,-0.308820,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20 Microns Ltd.,2008-10-08,13.25,-0.119601,0.023232,-3.533362,-3.510130,-1.780674,0.072932,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.349361,0.091985,-11.254830,-3.066034,0.612259,NaN,0.228562,-12.593540,-5.876183,0.191663,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20 Microns Ltd.,2008-10-10,11.60,-0.124528,0.045520,-7.052324,-7.006804,0.217126,0.354629,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.182061,0.135616,-5.542948,-4.000644,0.573414,-0.388754,0.181132,-5.036699,-4.725787,1.836919,NaN,0.273128,-16.594178,-5.339991,1.320939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20 Microns Ltd.,2008-10-13,12.32,0.062069,0.066863,5.042738,5.109602,-2.437097,0.145853,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.163920,0.134667,-0.834193,-1.442626,1.579289,-0.183251,0.180182,-9.543586,-3.464452,1.702691,-0.163688,0.225697,-12.320480,-5.095004,1.701136,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13465088,Zylog Systems Ltd.,2024-03-21,0.35,0.000000,0.018215,1.441807,1.460021,0.572866,1.270303,-0.026511,0.002663,0.002803,-0.002113,0.025498,0.008887,0.207118,0.000000,0.054643,2.222872,0.647351,1.691011,0.000000,0.145734,0.669402,-0.213495,1.701467,0.000000,0.218599,1.086124,-0.832943,1.912107,NaN,NaN,NaN,NaN,NaN,0.020729,-0.010770,-0.029542,NaN
13465089,Zylog Systems Ltd.,2024-03-22,0.35,0.000000,0.018215,0.593324,0.611538,0.752002,0.124527,-0.026262,0.002406,0.002773,-0.002086,0.024682,0.008058,0.220363,0.000000,0.109307,1.990617,0.233272,1.835435,0.000000,0.145735,2.505059,-0.609317,1.943111,0.000000,0.182163,1.699158,-0.957902,1.419960,NaN,NaN,NaN,NaN,NaN,-0.032129,-0.014710,0.003200,NaN
13465090,Zylog Systems Ltd.,2024-03-26,0.35,0.000000,0.072878,-0.044514,0.028364,-1.091597,0.440605,-0.026266,0.002386,0.002795,-0.002064,0.024686,0.008062,0.220298,0.000000,0.109307,0.875511,-0.504666,0.376627,0.000000,0.145736,3.020372,-0.511135,1.553929,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.030409,-0.016975,NaN,NaN
13465091,Zylog Systems Ltd.,2024-03-27,0.35,0.000000,0.018215,0.326702,0.344916,-0.165072,-0.188505,-0.026598,0.002727,0.003744,-0.001882,0.035403,0.018961,0.095276,0.000000,0.109307,0.985242,-1.836003,0.159099,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.025026,NaN,NaN,NaN


In [35]:
output180CAR_2 = output180CAR.dropna(subset = "180CAR3").sort_values( by = ["CompanyName", "AsOnDate"] ).reset_index(drop = True)

In [36]:
output180CAR_2

,CompanyName,AsOnDate,ACP,pct,RF,RMRF,MF,SMB,HML,OLS180_intercept,OLS180_RMRF,OLS180_SMB,OLS180_HML,OLS180_r_squared,OLS180_adjusted_r_squared,OLS180_f_p_value,pct_cen3sum,RF_cen3sum,RMRF_cen3sum,SMB_cen3sum,HML_cen3sum,pct_cen5sum,RF_cen5sum,RMRF_cen5sum,SMB_cen5sum,HML_cen5sum,pct_cen7sum,RF_cen7sum,RMRF_cen7sum,SMB_cen7sum,HML_cen7sum,pct_cen11sum,RF_cen11sum,RMRF_cen11sum,SMB_cen11sum,HML_cen11sum,180CAR3,180CAR5,180CAR7,180CAR11
0,20 Microns Ltd.,2009-07-22,11.60,-0.004292,0.008824,-0.838877,-0.830052,1.315390,-0.113483,-0.021264,0.009618,0.012246,0.004820,0.191707,0.177851,3.880499e-08,0.025918,0.026473,0.666990,1.985776,0.413217,0.053112,0.061772,4.334099,2.432023,0.461691,0.157955,0.097071,7.550620,2.937136,0.181165,0.102805,0.132152,11.307014,4.450350,0.387104,0.030511,0.023964,0.100263,0.039433
1,20 Microns Ltd.,2009-07-23,11.85,0.021552,0.008824,2.213316,2.222140,-0.356213,-0.135894,-0.021177,0.009572,0.012166,0.004828,0.191666,0.177887,3.513662e-08,0.038357,0.026473,2.731455,1.269163,0.786902,0.140403,0.061772,2.477478,3.200545,1.731442,0.115511,0.097071,5.397772,4.402621,0.394980,0.094868,0.132260,7.973263,4.542449,-0.398697,0.030030,0.113504,0.059542,0.065898
2,20 Microns Ltd.,2009-07-24,12.10,0.021097,0.008824,1.357015,1.365840,0.309985,1.036279,-0.020713,0.009391,0.011592,0.005322,0.187514,0.173665,5.458318e-08,0.136037,0.044123,4.023804,0.858557,1.182330,0.100755,0.061772,3.795128,3.239760,0.720191,0.092253,0.079420,1.900493,4.546625,1.163520,0.127467,0.132260,8.514635,3.551244,-0.450222,0.100021,0.065523,0.081083,0.104323
3,20 Microns Ltd.,2009-07-27,13.23,0.093388,0.026475,0.453473,0.479948,0.904784,0.281945,-0.020111,0.008978,0.010864,0.005588,0.171890,0.157774,2.800550e-07,0.083495,0.044123,2.420688,2.280583,0.969568,0.087886,0.061772,3.446820,2.204637,0.614409,0.075658,0.079420,3.214546,3.469058,0.577649,0.141755,0.149587,7.260741,3.507209,1.516315,0.047778,0.068341,0.067241,0.101629
4,20 Microns Ltd.,2009-07-28,12.82,-0.030990,0.008824,0.610200,0.619024,1.065814,-0.348656,-0.019722,0.008387,0.010366,0.007027,0.157005,0.142635,1.287276e-06,0.045238,0.044123,-0.123512,2.250864,-0.285976,0.058398,0.061772,1.840107,2.509881,0.827026,0.105550,0.079312,4.987819,1.472665,0.550944,0.177482,0.131828,4.508335,3.984572,3.244338,0.039994,0.047974,0.103321,0.160682
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12772601,Zylog Systems Ltd.,2024-03-20,0.35,0.000000,0.018213,0.187741,0.205954,-0.677517,0.296181,-0.026544,0.002821,0.002980,-0.002220,0.028359,0.011797,1.662029e-01,0.000000,0.054642,0.120592,0.126100,1.136335,0.000000,0.127506,0.803937,0.423725,1.660006,0.000000,0.218598,0.390646,0.786768,1.386466,0.000000,0.291507,0.039579,-0.033743,0.057616,0.026798,0.005370,-0.033155,0.000599
12772602,Zylog Systems Ltd.,2024-03-21,0.35,0.000000,0.018215,1.441807,1.460021,0.572866,1.270303,-0.026511,0.002663,0.002803,-0.002113,0.025498,0.008887,2.071184e-01,0.000000,0.054643,2.222872,0.647351,1.691011,0.000000,0.145734,0.669402,-0.213495,1.701467,0.000000,0.218599,1.086124,-0.832943,1.912107,NaN,NaN,NaN,NaN,NaN,0.020729,-0.010770,-0.029542,NaN
12772603,Zylog Systems Ltd.,2024-03-22,0.35,0.000000,0.018215,0.593324,0.611538,0.752002,0.124527,-0.026262,0.002406,0.002773,-0.002086,0.024682,0.008058,2.203633e-01,0.000000,0.109307,1.990617,0.233272,1.835435,0.000000,0.145735,2.505059,-0.609317,1.943111,0.000000,0.182163,1.699158,-0.957902,1.419960,NaN,NaN,NaN,NaN,NaN,-0.032129,-0.014710,0.003200,NaN
12772604,Zylog Systems Ltd.,2024-03-26,0.35,0.000000,0.072878,-0.044514,0.028364,-1.091597,0.440605,-0.026266,0.002386,0.002795,-0.002064,0.024686,0.008062,2.202977e-01,0.000000,0.109307,0.875511,-0.504666,0.376627,0.000000,0.145736,3.020372,-0.511135,1.553929,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.030409,-0.016975,NaN,NaN


In [37]:
output180CAR_2.to_csv("output180CAR.csv")